In [1]:
import numpy as np 
from scipy.io import loadmat
import os
from os.path import join as opj
from h5py import File
import pandas as pd

# load CLIP from huggingface, load the first N images and extract the features
from transformers import CLIPProcessor, CLIPModel
import torch
import tqdm
from PIL import Image

/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_path ="/home/matteo/storage/THINGS_Monkey"
monkey = "F"
os.listdir(base_path)

['THINGS_normMUA_F.mat',
 'THINGS_normMUA_N.mat',
 'THINGS_MUA_trials_N.mat',
 'THINGS_MUA_trials_F.mat',
 'THINGS_normMUA.mat',
 'things_imgs_F.mat']

In [3]:
things_imgs = File(opj(base_path,f'things_imgs_{monkey}.mat'))
train_imgs = things_imgs['train_imgs']   # group object --> <HDF5 group "/train_imgs" (3 members)>
test_imgs = things_imgs['test_imgs']


In [4]:
def resolve_reference(hdf5_file, ref):
    """
    Resolve an HDF5 dataset reference and convert it into a string.
    """
    data = hdf5_file[ref][:]
    return ''.join(chr(i) for i in data.flatten() if i > 0)

In [5]:
train_classes = []
train_local_paths = []
train_things_paths = []

test_classes = []
test_local_paths = []
test_things_paths = []

with File(opj(base_path, f"things_imgs_{monkey}.mat")) as f:
    train_imgs = f['train_imgs']
    
    train_classes = [resolve_reference(things_imgs, ref[0]) for ref in things_imgs['train_imgs']['class']]
    train_local_paths = [resolve_reference(things_imgs, ref[0]) for ref in things_imgs['train_imgs']['local_path']]
    train_things_paths = [resolve_reference(things_imgs, ref[0]) for ref in things_imgs['train_imgs']['things_path']]

    test_classes = [resolve_reference(things_imgs, ref[0]) for ref in things_imgs['test_imgs']['class']]
    test_local_paths = [resolve_reference(things_imgs, ref[0]) for ref in things_imgs['test_imgs']['local_path']]
    test_things_paths = [resolve_reference(things_imgs, ref[0]) for ref in things_imgs['test_imgs']['things_path']]

In [8]:
trials = File(opj(base_path, f"THINGS_MUA_trials_{monkey}.mat"))
df =pd.DataFrame(trials["ALLMAT"][:].T, columns=["#trial_idx", "#train_idx", "#test_idx", "#rep", "#count", "#correct"])

df

,#trial_idx,#train_idx,#test_idx,#rep,#count,#correct
0,1.0,16504.0,0.0,1.0,1.0,1.0
1,2.0,16470.0,0.0,1.0,2.0,1.0
2,3.0,15094.0,0.0,1.0,3.0,1.0
3,4.0,2514.0,0.0,1.0,4.0,1.0
4,5.0,4860.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...
25243,25244.0,0.0,40.0,30.0,1.0,4.0
25244,25245.0,13906.0,0.0,1.0,2.0,4.0
25245,25246.0,470.0,0.0,1.0,3.0,4.0
25246,25247.0,15559.0,0.0,1.0,4.0,4.0


In [9]:
# np.save('/srv/nfs-data/sisko/matteoc/monkeys/trials_allmua.npy', data)
data = np.load('/srv/nfs-data/sisko/matteoc/monkeys/trials_allmua.npy')

print(data.shape)

(300, 25248, 1024)


## Extract Images

In [10]:
thing_base_path ="/home/matteo/storage/THINGS_img/"
N = 15000      # TODO: prenderli tutti (dati training)

train_indices = df[df["#train_idx"]!=0]["#train_idx"].values.astype(int) - 1
test_indices = df[df["#test_idx"]!=0]["#test_idx"].values.astype(int) - 1

sorted_train_img_path = [train_things_paths[i] for i in train_indices]
sorted_test_img_path = [test_things_paths[i] for i in test_indices]

In [12]:
device = "cuda:3" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def extract_features(model, images):
    images = [Image.open(img).convert("RGB") for img in images]
    inputs = processor(images= images, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    return outputs

batch = 256
train_features = []
test_features = []

for i in tqdm.trange(0, N, batch):
    features = extract_features(model, [opj(thing_base_path,"THINGS","Images", img).replace("\\","/") for img in sorted_train_img_path[i:i+batch]])
    train_features.append(features.cpu().numpy())

for i in tqdm.trange(0, len(sorted_test_img_path), batch):
    features = extract_features(model, [opj(thing_base_path,"THINGS","Images", img).replace("\\","/") for img in sorted_test_img_path[i:i+batch]])
    test_features.append(features.cpu().numpy())

train_features = np.concatenate(train_features, axis=0)[:N]
test_features = np.concatenate(test_features, axis=0)


/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
100%|██████████| 12/12 [00:41<00:00,  3.42s/it]


In [11]:
# np.save('/srv/nfs-data/sisko/matteoc/monkeys/train_features.npy', train_features)
# np.save('/srv/nfs-data/sisko/matteoc/monkeys/test_features.npy', test_features)

train_features = np.load('/srv/nfs-data/sisko/matteoc/monkeys/train_features.npy')
test_features = np.load('/srv/nfs-data/sisko/matteoc/monkeys/test_features.npy')

In [12]:
train_features.shape, test_features.shape

((15000, 512), (3000, 512))

In [ ]:
neural_train_trial_idx = df[df["#train_idx"]!=0]["#trial_idx"].values.astype(int) - 1
neural_test_trial_idx = df[df["#test_idx"]!=0]["#trial_idx"].values.astype(int) - 1

train_neural = data[:,neural_train_trial_idx[:N]]     
test_neural = data[:,neural_test_trial_idx]
print(train_neural.shape, test_neural.shape)

(300, 15000, 1024) (300, 3000, 1024)


## Avg activity over repetitions

In [14]:
test_neural_avg = []
test_features_avg = []
selected_test_imgs = []
for idx in tqdm.tqdm(np.unique(test_indices)):
    test_neural_avg.append(test_neural[:,test_indices==idx].mean(1))
    #same for the test features
    test_features_avg.append(test_features[test_indices==idx].mean(0))
    selected_test_imgs.append(np.array(sorted_test_img_path)[test_indices==idx][0])

test_neural_avg = np.array(test_neural_avg).transpose(1,0,-1)
test_features_avg = np.array(test_features_avg)
selected_test_imgs = np.array(selected_test_imgs)

print(test_neural_avg.shape, test_features_avg.shape, selected_test_imgs.shape)

100%|██████████| 100/100 [00:01<00:00, 84.42it/s]

(300, 100, 1024) (100, 512) (100,)


In [15]:
test_neural_avg.shape, test_features_avg.shape, train_features.shape, train_neural.shape     # TODO: primi 100 di prestimolo

((300, 100, 1024), (100, 512), (15000, 512), (300, 15000, 1024))